## 半正定规划（semi-definite programming）的求解算法介绍

写作这一篇notebook的主要动机有以下两点：
* 通过具体代码实现来深入理解SDP问题求解算法的基本原理；
* 对已有求解器在实际求解中的好用程度进行评估选择。对求解器的计算效率比较可以参考[Hans Mittlemann教授的benchmarking](http://plato.asu.edu/ftp/sparse_sdp.html)；但是在实际使用过程中，API的清晰程度、与其他工具包交互的难易程度、问题数据读写的效率等因素也很重要。

首先，我们进行一些准备工作：从[这个网址](http://euler.nmt.edu/~brian/sdplib/)中获取SDPLIB测试案例，用于后续对求解器的评估测试。

In [1]:
import glob

fnames = sorted(glob.glob('sdplib/*.dat-s'))
print(len(fnames))

92


### SDP问题定义

原始问题
$$
\begin{equation}\label{eq:P}
    \begin{aligned}
        \max \ & C(X) \\
        s.t. \ & A(X) = b \\
             \ & X \succeq 0
    \end{aligned} \tag{P}
\end{equation}
$$

其中$A(X) = [\text{tr}(A_1X),\cdots,\text{tr}(A_nX)]^T$；其对偶问题为

$$
\begin{equation}\label{eq:D}
    \begin{aligned}
        \min \ & b^Ty \\
        s.t. \ & A^T(y) \succeq C \\
    \end{aligned} \tag{D}
\end{equation}
$$

其中$A^T(y) = \sum_{i=1}^n y_i A_i$。

### 调用已有求解器

首先，我们介绍一些已有求解器的使用方法。这里，我们主要介绍三种求解器：

1. [CSDP](https://github.com/coin-or/Csdp)：用C语言实现的SDP求解器。
    * 安装：从[git repo](https://github.com/coin-or/Csdp)中获取源代码，然后根据本地机器的配置修改makefile，手动编译。
    * 使用：需要通过命令行执行；Python可以通过subprocess进行调用。一个缺点是对于Python脚本产生的问题数据，需要先写成一个数据文件、然后再调用CSDP，存在IO上的增量计算开销。这在需要resolve的场景中不是很友好。
2. [CVXOPT](https://github.com/cvxopt/cvxopt)：面向Python的通用凸优化求解器，对不同类型的凸优化问题提供了一套统一的建模和求解接口。
    * 安装：可以直接pip install安装。
    * 使用：自身提供的API可以直接处理内存中的问题数据；如果是数据文件，则需要手动读取或者使用smcp提供的读取函数。
3. [smcp](https://github.com/cvxopt/smcp)：用C和Python实现的SDP求解器，是CVXOPT项目的一部分。
    * 安装：可以直接pip install安装。
    * 使用：自身提供的API可以处理数据文件或是内存中的问题数据。

下面的代码块将通过SDPLIB中的测试案例，对上述求解器的求解正确性和计算效率进行评估比较。

In [2]:
import smcp
import subprocess
from IPython.utils import io
import time
import numpy as np

INF = 1e20

for fname in fnames:
    if 'G' not in fname:
        model_name = fname.split('.')[0].split('/')[1]
        out_fname = 'outputs/{}.out'.format(model_name)

        with io.capture_output() as captured:
            tt = time.time()
            ## csdp
            obj_csdp = -INF
            try:
                subprocess.call('csdp {} > {}'.format(fname,out_fname),
                               shell=True,stdout=subprocess.PIPE)
                with open(out_fname,'r+') as f:
                    lines = f.readlines()
                    for line in lines:
                        if 'Primal objective value:' in line:
                            obj_csdp = float(line[24:].strip())
                            break
            except Exception as e:
                print(e)
            time_csdp = time.time() - tt

            sdp = smcp.SDP(fname)
            tt = time.time()
            ## smcp
            obj_smcp = -INF
            try:
                sol_smcp = sdp.solve_esd()
                obj_smcp = -sol_smcp['primal_objective']
            except Exception as e:
                print(e)
            time_smcp = time.time() - tt

            tt = time.time()
            ## cvx
            obj_cvx = -INF
            try:
                sol_cvx = sdp.solve_cvxopt(options={'show_progress':False, 'maxiters':50})
                obj_cvx = sol_cvx['primal objective']
            except Exception as e:
                print(e)
            time_cvx = time.time() - tt
            
        print('elapsed time: csdp={:.2f}, smcp={:.2f}, cvx={:.2f}'.format(time_csdp,time_smcp,time_cvx))
        print('objectives: csdp={:.2f}, smcp={:.2f}, cvx={:.2f}'.format(obj_csdp,obj_smcp,obj_cvx))

根据上面的结果可以看出，CSDP显著更快；这是由于CSDP是针对SDP问题而专门开发的求解器。CVXOPT则作为通用的凸优化求解器，计算效率较低。最后，smcp经常得不到正确结果，因此不建议使用。

### 内点法的实现

在这一小节中，我将实现一种基础的求解方法：引入罚函数的内点法。具体来说，我们可以在对偶问题中引入关于$S = C - A^T(y)$惩罚项，从而提高问题的光滑程度并降低求解难度：
$$
\begin{equation}\label{eq:DB}
    \begin{aligned}
        \min \ & b^Ty - \mu \log\det S \\
        s.t. \ & A^T(y) = S + C \\
             \ & S \succeq 0
    \end{aligned} \tag{DB}
\end{equation}
$$

其中$\mu \geq 0$是一个超参数，当$\mu \to 0$时，上述问题的最优解趋于原对偶问题的最优解。上述问题的KKT条件为
$$
\begin{equation}\label{eq:KKT}
    \begin{aligned}
        \ & A(x) = b \\
        \ & A^T(y) = S + C \\
        \ & SX = \mu I \\
        \ & X \succeq 0, S \succeq 0
    \end{aligned} \tag{KKT}
\end{equation}
$$

我们可以通过牛顿法来求解该KKT条件。具体来说，给定一个超参数$\mu$和一个初始解$(X,S,y)$（满足$X$和$S$是半正定的），上述KKT系统在该初始解的线性化形式为
$$
\begin{equation}\label{eq:KKT_linear}
    \begin{aligned}
        \ & A(X + \Delta X) = b \\
        \ & A^T(y + \Delta y) = S + \Delta S + C \\
        \ & SX + \Delta S X + S \Delta X = \mu I
    \end{aligned} \tag{KKT linearization}
\end{equation}
$$

要求解该线性系统，首先，将第二个等式中$\Delta S$的形式代入第三个等式中，有
$$
\begin{equation}
    \begin{aligned}
        X + \Delta X & = - S^{-1} (\Delta S X - \mu I) \\
        & = \tilde{C} - S^{-1} A^T(\Delta y) X \\
    \end{aligned}
\end{equation}
$$

其中$\tilde{C} = - S^{-1} ((A^Ty - S - C) X - \mu I)$。然后，再将上述式子代入线性系统的第一个等式，有
$$
\begin{equation}
    \begin{aligned}
        A(\tilde{C}) - A(S^{-1} A^T(\Delta y) X) & = b \\
    \end{aligned}
\end{equation}
$$

换句话说，如果我们令$O$为由元素$O_{ij} = \text{tr}(A_i S^{-1} A_j X)$组成的矩阵，那么上述等式可以写为
$$
\begin{equation}
    O \Delta y = A(\tilde{C}) - b
\end{equation}
$$

此时，我们可以通过任意线性方程求解器来求解得到$\Delta y$。然后通过回代可以得到$\Delta X$和$\Delta S$。

最后，$X + \Delta X$或$S + \Delta S$有可能不是半正定的，这会导致迭代过程无法继续下去。因此我们需要寻找合适的步长$\alpha_p$和$\alpha_d$使得$X + \Delta X$或$S + \Delta S$都是半正定的。由于$X$和$S$是对称阵，因此可以通过找到$X^{-1/2}\Delta X X^{-1/2}$和$S^{-1/2}\Delta S S^{-1/2}$的最大特征值来设计步长。

下面的代码块给出了上述算法的一个简单实现，其中一些辅助工作作为函数被封装在了util.py中：
* process_inputs函数从数据文件中读取问题数据。它先使用smcp包中的sdpa_read函数对.dat-s文件(SDPA格式)进行读取，然后将问题输入数据转化为scipy的稀疏矩阵格式。
* initialization函数根据问题输入$A,C,b$的结构初始化矩阵$X,S$。这里的实现参考了CSDP中的initsoln函数，但是没有利用矩阵$A,C$的分块特征。
* line_search函数负责寻找合适的迭代步长。它首先根据矩阵$X$和增量$\Delta X$，尝试找到最大可行的步长$\alpha_{max} > 0$使得$X + \alpha \Delta X$是半正定的。如上所述，这可以转化为一个特征值求解问题。在找到最大可行步长$\alpha_{max}$后，令实际步长$\alpha$为$\alpha_{max}$的一定比例的折减（例如，$\alpha = 0.95 \alpha_{max}$），以使得更新后的矩阵$X + \alpha \Delta X$与半正定边界保持一定距离，不至于影响后续迭代的效率。

In [3]:
import time
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csr_matrix,csc_matrix
import scipy.sparse.linalg as splinalg
import smcp
from sksparse.cholmod import cholesky,Factor
from util import *

def solve(fname,print_iter = 100):
    model_name = fname.split('.')[0].split('/')[1]
    
    ## load inputs
    n,m,C_csc,b_dense,As_h,As_csr,As_csc = process_inputs(fname,model_name)
    
    total_tt = time.time()
    
    ## initialization
    I = sp.eye(n,n,dtype=np.float64).tocsc()
    X,S = initialization(As_csc,C_csc,b_dense,n,m)
    y = np.zeros((m,))
    S_inv_op = cholesky(S)
    mu = np.sum((S.multiply(X)).data) / n / 2 ## tr(SX)
    
    iter_ = 0
    while iter_ < 10000 and (time.time() - total_tt) < 1200:
        iter_ += 1
        tt = time.time()
        
        ## compute objectives and errors in constraints
        obj_primal = np.sum((C_csc.multiply(X)).data) ## C(X)
        obj_dual = np.dot(b_dense,y) ## b^Ty
        ep_dual = csc_matrix(As_csc.dot(y).reshape((n,n))) - S - C_csc ## A^T(y) - S - C
        ep_primal = X.reshape((n*n,1)).T.dot(As_csc).A[0,:] - b_dense ## A(X) - b
        ep_comp = S.dot(X) - mu * I ## SX - \mu I
        err_primal = np.max(np.abs(ep_primal))
        if len(ep_dual.data) > 0:
            err_dual = np.max(np.abs(ep_dual.data))
        else:
            err_dual = 0
        if len(ep_comp.data) > 0:
            err_comp = np.max(np.abs(ep_comp.data))
        else:
            err_comp = 0
        if iter_ % print_iter == 0:
            print('iter={}. objp={:.3e}, objd={:.3e}, errp={:.3e}, errd={:.3e}, errcs={:.3e}'.format(
                iter_,obj_primal,obj_dual,err_primal,err_dual,err_comp))
            
        ## check for convergence
        is_converged = abs(obj_primal - obj_dual) < max(abs(obj_primal),abs(obj_dual),1) * TOL
        is_feas = (err_primal < TOL) and (err_dual < TOL)
        if is_converged and is_feas:
            break
            
        ## compute O
        SiAs_csc = S_inv_op.solve_A(As_h) ## S^{-1}A
        SiAs_csc = SiAs_csc.reshape((n*m,n))
        SiAXs_csc = SiAs_csc.dot(X) ## S^{-1}AX
        SiAXs_csc = SiAXs_csc.reshape((n,m*n))
        O = As_csr.dot(SiAXs_csc.reshape((n*n,m),order='F')).A
        
        ## compute v = A(C_tilde) - b
        C_tilde = -S_inv_op.solve_A(ep_dual.dot(X) - mu * I)
        v = (C_tilde.reshape((n*n,1)).T.dot(As_csc).A[0,:]-b_dense)
        
        ## solve dy = O^{-1} v
        try:
            dy = np.linalg.solve(O,v)
        except Exception:
            print('O near psd boundary! giving up.')
            break
            
        ## solve dS = A^T(dy) + (A^T(y) - S - C)
        dS = ep_dual + csc_matrix(As_csc.dot(dy).reshape((n,n)))
        ## solve dX = S^{-1}(dS X + SX - \mu I)
        dX = S_inv_op.solve_A(-(dS.dot(X) + ep_comp))
        dX = (dX + dX.T) / 2
        
        ## line search for step size
        alpha_p = line_search(X,dX)
        alpha_d = line_search(S,dS)
        if alpha_p < 0 or alpha_d < 0:
            psd_p,psd_d = 0,0
            try:
                psd_p,psd_d = np.min(cholesky(X).D()),np.min(cholesky(S).D())
            except Exception as e:
                pass
            print('X/S near psd boundary! giving up. '
                  'psd of primal={:.4e}, psd of dual={:.4e}.'.format(
                    psd_p,psd_d))
            break
            
        ## update the solution
        X = X + alpha_p * dX
        y = y + alpha_d * dy
        S = S + alpha_d * dS
        S_inv_op = cholesky(S)
        mu = np.sum((S.multiply(X)).data) / n / 2 ## tr(SX)
        if alpha_p + alpha_d > 1.8: ## good progress, so be more aggresive next step
            mu /= 2
            
        if iter_ % print_iter == 0:
            print('iter={}. elapsed time={:.2f}. mu={:.3f}, alphap={:.3f}, alphad={:.3f}.'.format(
                    iter_,time.time()-tt,mu,alpha_p,alpha_d))
            
    print('{}: total iter={}, elapsed time={:.2f}. objp={:.3e}, objd={:.3e}, '
          'errp={:.3e}, errd={:.3e}, errcs={:.3e}'.format(
            model_name,iter_,time.time()-total_tt,obj_primal,obj_dual,err_primal,err_dual,err_comp))

接下来，我们通过SDPLIB中的测试案例来检验上述算法的正确性和计算效率。

In [4]:
for fname in fnames:
    if 'G' not in fname:
        solve(fname,print_iter=100)

arch0: model finish reading. n=335, m=174, nnz=3030.
initialization scale: p=10.00, d=10.00.


/Users/hanqiu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: CholmodTypeConversionWarning: converting matrix of class coo_matrix to CSC format
/Users/hanqiu/Documents/GitHub/sdp_solve/util.py:109: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  PdXPt = factor.apply_P(factor.apply_P(dX).T)
/Users/hanqiu/Documents/GitHub/sdp_solve/util.py:110: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  LiPdXPtLit = factor.solve_L(factor.solve_L(PdXPt,use_LDLt_decomposition=False).T,use_LDLt_decomposition=False)


iter=100. objp=5.665e-01, objd=5.665e-01, errp=3.319e-04, errd=4.263e-14, errcs=1.168e-05
iter=100. elapsed time=0.37. mu=0.000, alphap=0.000, alphad=0.019.
X/S near psd boundary! giving up. psd of primal=0.0000e+00, psd of dual=0.0000e+00.
arch0: total iter=141, elapsed time=51.33. objp=5.665e-01, objd=5.665e-01, errp=6.118e-04, errd=4.263e-14, errcs=1.150e-05
arch2: model finish reading. n=335, m=174, nnz=3420.
initialization scale: p=10.00, d=10.00.
X/S near psd boundary! giving up. psd of primal=0.0000e+00, psd of dual=0.0000e+00.
arch2: total iter=75, elapsed time=26.45. objp=6.715e-01, objd=6.715e-01, errp=2.678e-04, errd=2.842e-14, errcs=6.878e-06
arch4: model finish reading. n=335, m=174, nnz=3420.
initialization scale: p=10.00, d=10.00.
iter=100. objp=9.726e-01, objd=9.726e-01, errp=2.557e-04, errd=2.842e-14, errcs=5.551e-06
iter=100. elapsed time=0.34. mu=0.000, alphap=0.002, alphad=0.084.
X/S near psd boundary! giving up. psd of primal=0.0000e+00, psd of dual=0.0000e+00.
arc

control8: total iter=94, elapsed time=146.79. objp=2.029e+01, objd=2.029e+01, errp=8.843e-07, errd=9.313e-10, errcs=2.314e-03
control9: model finish reading. n=135, m=1081, nnz=229005.
initialization scale: p=10.00, d=10.00.
control9: total iter=84, elapsed time=223.64. objp=1.468e+01, objd=1.468e+01, errp=5.516e-07, errd=9.313e-10, errcs=1.575e-03
gpp100: model finish reading. n=100, m=101, nnz=5150.
initialization scale: p=10.00, d=15.31.
X/S near psd boundary! giving up. psd of primal=0.0000e+00, psd of dual=0.0000e+00.
gpp100: total iter=73, elapsed time=6.90. objp=-4.506e+01, objd=-4.494e+01, errp=7.086e-05, errd=2.980e-08, errcs=2.255e+03
gpp124-1: model finish reading. n=124, m=125, nnz=7874.
initialization scale: p=10.00, d=10.00.
X/S near psd boundary! giving up. psd of primal=0.0000e+00, psd of dual=0.0000e+00.
gpp124-1: total iter=25, elapsed time=4.13. objp=-7.441e+00, objd=-7.341e+00, errp=2.244e-05, errd=1.863e-09, errcs=1.664e+02
gpp124-2: model finish reading. n=124, m=

iter=400. objp=2.397e+01, objd=2.396e+01, errp=3.834e-02, errd=1.097e-05, errcs=6.189e+04
iter=400. elapsed time=0.02. mu=0.005, alphap=0.000, alphad=0.000.
iter=500. objp=2.397e+01, objd=2.396e+01, errp=3.829e-02, errd=1.049e-05, errcs=5.149e+04
iter=500. elapsed time=0.01. mu=0.005, alphap=0.000, alphad=0.000.
iter=600. objp=2.397e+01, objd=2.396e+01, errp=3.826e-02, errd=1.001e-05, errcs=4.168e+04
iter=600. elapsed time=0.01. mu=0.005, alphap=0.000, alphad=0.000.
iter=700. objp=2.397e+01, objd=2.396e+01, errp=3.823e-02, errd=1.240e-05, errcs=1.840e+04
iter=700. elapsed time=0.01. mu=0.004, alphap=0.000, alphad=0.000.
iter=800. objp=2.397e+01, objd=2.396e+01, errp=3.820e-02, errd=1.192e-05, errcs=1.235e+04
iter=800. elapsed time=0.01. mu=0.004, alphap=0.000, alphad=0.000.
iter=900. objp=2.397e+01, objd=2.396e+01, errp=3.817e-02, errd=9.537e-06, errcs=1.211e+04
iter=900. elapsed time=0.01. mu=0.004, alphap=0.000, alphad=0.000.
iter=1000. objp=2.397e+01, objd=2.396e+01, errp=3.814e-02,

mcp500-2: total iter=23, elapsed time=212.50. objp=1.070e+03, objd=1.070e+03, errp=8.785e-10, errd=4.441e-16, errcs=6.837e-06
mcp500-3: model finish reading. n=500, m=500, nnz=500.
initialization scale: p=10.00, d=25.85.
mcp500-3: total iter=21, elapsed time=261.91. objp=1.848e+03, objd=1.848e+03, errp=6.502e-10, errd=8.882e-16, errcs=9.846e-06
mcp500-4: model finish reading. n=500, m=500, nnz=500.
initialization scale: p=10.00, d=53.51.
mcp500-4: total iter=19, elapsed time=318.95. objp=3.567e+03, objd=3.567e+03, errp=4.056e-10, errd=1.776e-15, errcs=5.298e-05
qap10: model finish reading. n=101, m=1021, nnz=13101.
initialization scale: p=10.00, d=2400.01.
X/S near psd boundary! giving up. psd of primal=0.0000e+00, psd of dual=0.0000e+00.
qap10: total iter=61, elapsed time=48.91. objp=-1.093e+03, objd=-1.093e+03, errp=1.873e-04, errd=6.985e-10, errcs=1.476e+00
qap5: model finish reading. n=26, m=136, nnz=1026.
initialization scale: p=10.00, d=590.54.
X/S near psd boundary! giving up. p

可以看出，上面的算法实现基本可以得到正确结果，但计算效率比CSDP慢10~100倍，这主要包含两方面原因：
1. 迭代效率：在不同测试问题上，上述实现的迭代数通常比CSDP高好几倍。这是由于上述实现在步长选取、迭代步更新（是否引入预估-校正步骤）等方面都使用了最基础的技术，相比CSDP经过一定优化的实现有明显差距；
2. 底层计算效率：虽然最终执行的都是经过优化的C代码，但上述实现是基于Python的，需要增加一定的语言转换开销。另外，在利用问题矩阵$A$和$C$的稀疏性和分块特征上，通用的线性代数函数也没有CSDP中专门实现的函数效率高。

### 参考文献

上面的实现只对应了SDP的一种求解方法，而且仅包括了最基本的技术。关于更多提高计算效率的技术，感兴趣的读者可以参考以下文章：
* 对不等式约束$B(X) \succeq b$的考虑：Helmberg, C., Rendl, F., Vanderbei, R. J., & Wolkowicz, H. (1996). An interior-point method for semidefinite programming. SIAM Journal on Optimization, 6(2), 342-361.
* 利用预估-校正(predictor-corrector)技术提高内点法迭代效率：Borchers, B. (1999). CSDP, A C library for semidefinite programming. Optimization methods and Software, 11(1-4), 613-623.
* 将问题转化为自对偶形式(self-dual)来更好地判断解的不可行性：Domahidi, A., Chu, E., & Boyd, S. (2013, July). ECOS: An SOCP solver for embedded systems. In 2013 European Control Conference (ECC) (pp. 3071-3076). IEEE.